In [1]:
# import rmu
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load model directly
print("Loading model...")
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2", output_hidden_states=True)

print(model)

/home/ubuntu/miniconda3/envs/wmdp_replication/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model...


/home/ubuntu/miniconda3/envs/wmdp_replication/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:774: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [2]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"
torch.cuda.empty_cache()

In [3]:

# activations = {}
# def hook_fn(module, input, output):
#   activations["transformer_block_output"] = output[0].detach()

# layer_idx = 5


# hook = model.transformer.h[layer_idx].register_forward_hook(hook_fn)

# input_text = "Hello, how are you today?"
# inputs = tokenizer(input_text, return_tensors="pt")
# print(inputs["input_ids"].shape)

# # Forward pass through the model
# with torch.no_grad():
#     _ = model(**inputs)
# print(activations["transformer_block_output"].shape)
# hook.remove()

In [4]:
print(tokenizer.vocab_size)

50257


In [7]:
class Model():
  def __init__(self, model, tokenizer, device, seed = 42):
    self.model = model.to(device)
    self.tokenizer = tokenizer
    self.device = device
    self.seed = seed
    torch.manual_seed(seed)
    self.activations = {}

  def hook_fn(self, module, input, output):
    self.activations["transformer_block_output"] = output[0].detach()
  
  def forward(self, inputs, layer_idx: int):
    # For gemma, use `model.model.layers`
    if layer_idx >= len(self.model.transformer.h):
      raise ValueError(f"Layer index {layer_idx} is out of bounds for the model. The model has {len(self.model.transformer.h)} layers.")
    try:
      hook = self.model.transformer.h[layer_idx].register_forward_hook(self.hook_fn)
      tokenized_inputs = self.tokenizer(inputs, return_tensors="pt").to(self.device)

      print("tokenized_inputs: ", tokenized_inputs)
      print("tokenized_inputs.input_ids: ", tokenized_inputs.input_ids)
      print("max tokenized_inputs.input_ids: ", torch.max(tokenized_inputs.input_ids))
      print("min tokenized_inputs.input_ids: ", torch.min(tokenized_inputs.input_ids))
      print("tokenizer vocab size: ", tokenizer.vocab_size)
      with torch.no_grad():
        _ = self.model(**tokenized_inputs)
    finally:
      hook.remove()
    return self.activations["transformer_block_output"]

mymodel = Model(model, tokenizer, "cuda")

activation_output = mymodel.forward("Hello, world!", 5)
print(activation_output.shape)



tokenized_inputs:  {'input_ids': tensor([[15496,    11,   995,     0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1]], device='cuda:0')}
tokenized_inputs.input_ids:  tensor([[15496,    11,   995,     0]], device='cuda:0')
max tokenized_inputs.input_ids:  tensor(15496, device='cuda:0')
tokenizer vocab size:  50257
torch.Size([1, 4, 768])


In [8]:
inputs = tokenizer("Hello, world!", return_tensors="pt").to("cuda")
print(inputs.input_ids.shape)
outputs = model(**inputs)
print(outputs.hidden_states[5].shape)
# print(outputs.hidden_states.shape)

torch.Size([1, 4])
torch.Size([1, 4, 768])


In [9]:
import torch
k = torch.randn(10)
k = k / torch.norm(k)
print(k)
print(torch.norm(k))


tensor([ 0.1263,  0.0483,  0.0880,  0.0864, -0.4214, -0.0699,  0.8286, -0.2394,
         0.1732,  0.1003])
tensor(1.0000)


In [10]:
from datasets import load_dataset
from torch.utils.data import DataLoader

cyber_retain_dataset = load_dataset("json", data_files="data/cyber-retain-corpus.jsonl")
cyber_torch_retain_dataset = cyber_retain_dataset["train"].with_format("torch")
cyber_retain_dataloader = DataLoader(cyber_torch_retain_dataset, batch_size=1, shuffle=True)

cyber_forget_dataset = load_dataset("json", data_files="data/cyber-forget-corpus.jsonl")
cyber_torch_forget_dataset = cyber_forget_dataset["train"].with_format("torch")
cyber_forget_dataloader = DataLoader(cyber_torch_forget_dataset, batch_size=1, shuffle=True)


ctx_length = 512

# torch.manual_seed(42)

# for batch in cyber_retain_dataloader:
#   inputs = tokenizer(batch["text"], return_tensors="pt", truncation=True, max_length=ctx_length)
#   if inputs.input_ids.shape[1] > ctx_length:
#     inputs.input_ids = inputs.input_ids[:, :ctx_length]
#     inputs.attention_mask = inputs.attention_mask[:, :ctx_length]
#   outputs = model(**inputs)
#   break
  


In [11]:
# We want a Dataset class that loads in a jsonl file, tokenizes the dataset as expected and returns the input ids and attention mask.
import os
import json

class JsonlDataset():
  def __init__(self, tokenizer, tokenizer_max_length, batch_size, min_len, dataset_name, dataset_folder, device):
    self.tokenizer = tokenizer
    if tokenizer.pad_token is None:
      tokenizer.pad_token = tokenizer.eos_token

    self.tokenizer_max_length = tokenizer_max_length
    self.batch_size = batch_size
    self.min_len = min_len
    self.dataset_name = dataset_name
    self.dataset_folder = dataset_folder
    self.data = []
    self.device = device

  def __getitem__(self, idx):
    item = self.data[idx]
    input_ids = self.tokenizer(item["text"], return_tensors="pt", padding=True, truncation=True, max_length=self.tokenizer_max_length)
    inputs = {key: value.to(self.device) for key, value in input_ids.items()}
    return {"input_ids": inputs["input_ids"].squeeze(0), "attention_mask": inputs["attention_mask"].squeeze(0)}

  def _load_dataset(self):
    dataset_path = os.path.join(self.dataset_folder, self.dataset_name)
    if not os.path.exists(dataset_path):
      raise FileNotFoundError(f"Dataset file not found at {dataset_path}")
    
    data_list = []
    with open(dataset_path, "r") as f:
      for line in f:
        data = json.loads(line)
        if len(data["text"]) > self.min_len:
          data_list.append(data)

    self.data = data_list
    
  def __len__(self):
    return len(self.data)



In [12]:
mydataset = JsonlDataset(tokenizer=tokenizer, tokenizer_max_length=1024, batch_size=1, min_len=50, dataset_name="cyber-retain-corpus.jsonl", dataset_folder="data/", device="cuda")
mydataset._load_dataset()
print(len(mydataset))
print(mydataset[2])


4348
{'input_ids': tensor([  2,  58,  16,  ...,  11, 299, 532], device='cuda:0'), 'attention_mask': tensor([1, 1, 1,  ..., 1, 1, 1], device='cuda:0')}


In [13]:
import torch
import numpy as np

class RMU:
  def __init__(self, model, tokenizer, datasets, device, alpha, lr, c, hidden_dimension_size, ctx_window, min_len, seed = 42):
    self.unlearned_model = Model(model, tokenizer, device, seed)
    self.frozen_model = Model(model, tokenizer, device, seed)
    self.tokenizer = tokenizer
    self.datasets = datasets
    self.device = device
    self.alpha = alpha
    self.lr = lr
    self.c = c
    self.ctx_window = ctx_window
    self.min_len = min_len
    self.seed = seed
    self.hidden_dimension_size = hidden_dimension_size
    np.random.seed(seed)
    torch.manual_seed(seed)

    # Initialize u
    u = torch.randn(self.hidden_dimension_size)
    u = u / torch.norm(u)
    self.u = u

  def rmu_step(self, d_forget, d_retain, layer_idx):
    print("Beginning RMU step...")
    print("Creating cyber forget dataset...")
    cyber_forget = JsonlDataset(
      tokenizer=self.tokenizer, tokenizer_max_length=self.ctx_window, batch_size=1,
      min_len=self.min_len, dataset_name="cyber-forget-corpus.jsonl", dataset_folder="data/", device=self.device
    )
    cyber_forget._load_dataset()
    print("Finished creating cyber forget dataset.")
    print("Creating cyber retain dataset...")
    cyber_retain = JsonlDataset(
      tokenizer=self.tokenizer, tokenizer_max_length=self.ctx_window, batch_size=1,
      min_len=self.min_len, dataset_name="cyber-retain-corpus.jsonl", dataset_folder="data/", device=self.device
      )
    cyber_retain._load_dataset()
    print("Finished creating cyber retain dataset.")
  
    # TODO: Freeze the model parameters at a given layer
    for i in range(len(cyber_forget.data)):
      print("cyber forget text length: ", len(cyber_forget.data[i]["text"]))
      act_forget = self.unlearned_model.forward(cyber_forget.data[i]["text"], layer_idx)
      print("after act_forget but before act_retain")
      act_retain = self.unlearned_model.forward(cyber_retain.data[i]["text"], layer_idx)
      break
    
    print(act_forget.shape)
    print(act_retain.shape)

    print("Finished RMU step...")

print("Initializing RMU...")
myrmu = RMU(model=model, tokenizer=tokenizer, datasets=None, device="cuda", alpha=0.01, lr=0.001, c=1, hidden_dimension_size=768, min_len=50, ctx_window=1024)
print("Finished initializing RMU.")
myrmu.rmu_step(d_forget=cyber_torch_forget_dataset["text"], d_retain=cyber_torch_retain_dataset["text"], layer_idx=5)

Initializing RMU...
Finished initializing RMU.
Beginning RMU step...
Creating cyber forget dataset...
Finished creating cyber forget dataset.
Creating cyber retain dataset...


Token indices sequence length is longer than the specified maximum sequence length for this model (13868 > 1024). Running this sequence through the model will result in indexing errors


Finished creating cyber retain dataset.
cyber forget text length:  38706
tokenized_inputs:  {'input_ids': tensor([[   2,   58,   16,  ...,  337, 5258,  721]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], device='cuda:0')}
tokenized_inputs.input_ids:  tensor([[   2,   58,   16,  ...,  337, 5258,  721]], device='cuda:0')
max tokenized_inputs.input_ids:  tensor(50049, device='cuda:0')
tokenizer vocab size:  50257


/opt/conda/conda-bld/pytorch_1724789116784/work/aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [188,0,0], thread: [64,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1724789116784/work/aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [188,0,0], thread: [65,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1724789116784/work/aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [188,0,0], thread: [66,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1724789116784/work/aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [188,0,0], thread: [67,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda/conda-bld/pytorch_1724789116784/work/aten/src/ATen/native/cuda/Indexing.cu:1284: indexSelectLargeIndex: block: [188,0,0], thread: [68,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/opt/conda

RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
print(cyber_torch_forget_dataset)
print(cyber_torch_forget_dataset["text"])

In [ ]:
print("Initializing RMU...")
myrmu = RMU(model=model, tokenizer=tokenizer, datasets=None, device="cuda", alpha=0.01, lr=0.001, c=1, hidden_dimension_size=768, ctx_window=1024)

print("Running RMU step...")
myrmu.rmu_step(d_forget=cyber_torch_forget_dataset["text"], d_retain=cyber_torch_retain_dataset["text"], layer_idx=5)